## Import Libraries

In [1]:
!pip install transformers==4.2.2

     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 52.8 MB/s 
     |████████████████████████████████| 2.9 MB 51.7 MB/s 


In [2]:
!nvidia-smi

Fri Jan 28 22:46:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import pandas as pd


from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments,AutoModelForCausalLM
from transformers import pipeline

In [4]:
IN_COLAB: bool = False
try:
  from google.colab import drive, files
  IN_COLAB = True
except:
  IN_COLAB = False

path: str = ''
if IN_COLAB:
  drive.mount('/content/drive', force_remount=True)
  path = 'drive/MyDrive/RapMachine/KeywordLyrics.json'
else:
  path = None
df: pd.DataFrame = pd.read_json(path)

Mounted at /content/drive


In [5]:
df['lyrics'][0]

'\n"Yeah, Gangsta Dresta done stepped in this, motherfucker. And this one goes out to all the O.G.\'s out there. And I can\'t forget the baby gangstas, what\'s up, niggas? My nigga they came back with some of that old school original west coast shit, nigga. That got all of y\'all on his dick in the first place, yeah. So run that shit, E, yeah."Cruisin\' down the street in my \'64\nJockin\' the freaks, clockin\' the dough\nWent to the park to get the scoop\nKnuckleheads out there cold shooting some hoops\nA car pulls up, who can it be?\nA fresh El Camino rolling Kilo G\nHe rolls down his window and he started to say\nIt\'s all about making that GTACause the boys in the hood are always hard\nYou come talking that trash, we\'ll pull your card\nKnowing nothing in life, but to be legit\nDon\'t quote me, boy, cause I ain\'t said shitDonald B\'s in the place to give me the pace\nHe said, "My man JD is on freebase"\nThe boy JD was a friend of mine\n\'Til I caught him in my car trying to steal 

In [6]:
import re
import json
from sklearn.model_selection import train_test_split


def build_text_file(df: pd.DataFrame, dest_path: str) -> None:
  text: str = ''
  for _, row in df.iterrows():
    keywords: str = row[0]
    lyrics: str = row[1]
    # processed: str  = re.sub(r"\n", " ", lyrics.strip())
    # processed: str = re.sub(r'\n', '<|newline|>', lyrics)
    prompt: str = f"<|startoftext|>CONTEXT: {keywords} RAP-LYRICS: {lyrics}<|endoftext|>\n"
    text += prompt
    # text += processed
  f = open(dest_path, 'w')
  f.write(text)
  f.close()


train, test = train_test_split(df,test_size=0.15) 
build_text_file(train, 'drive/MyDrive/RapMachine/training_data/data_train.txt')
build_text_file(test, 'drive/MyDrive/RapMachine/training_data/data_test.txt')
pass

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    'distilgpt2',)
    # bos_token='<|startoftext|>',
    # eos_token='<|endoftext|>',)
    # additional_special_tokens=['<|newline|>'])

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator


train_path = 'drive/MyDrive/RapMachine/training_data/data_train.txt'
test_path = 'drive/MyDrive/RapMachine/training_data/data_test.txt'
train_dataset, test_dataset, data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (3053600 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
test_dataset

In [10]:
model_path: str = "drive/MyDrive/RapMachine/RM"

In [11]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [12]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained('distilgpt2')


training_args = TrainingArguments(
    output_dir="./gpt2-gerchef", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/353M [00:00<?, ?B/s]

In [13]:
model = AutoModelForCausalLM.from_pretrained('distilgpt2')
model.resize_token_embeddings(len(tokenizer))


training_args = TrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=1, #32, # batch size for training
    per_device_eval_batch_size=1, #64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [14]:
trainer.train()
trainer.save_model()

Step,Training Loss
500,4.286800
1000,4.038900
1500,4.015400
2000,3.992000
2500,3.871000
3000,3.906400
3500,3.856900
4000,3.871900
4500,3.811500
5000,3.840700


In [22]:
trainer.save_model('.')
tokenizer.save_pretrained('.')

('./tokenizer_config.json',
 './special_tokens_map.json',
 './vocab.json',
 './merges.txt',
 './added_tokens.json')

In [15]:
rapper = pipeline('text-generation',model=model_path, tokenizer='distilgpt2',config={'length': 200, 'max_length':1000, 'num_beam':2, 'do_sample': True, 'top_k':50})

In [16]:
input: str = 'CONTEXT: fore it crash, sky falling, Drop that ass, fore, higher, falling, Drop, crash, sky, ass, baby, switched my robot RAP-LYRICS:'
r = rapper(input)
output: str = str(r[0]['generated_text'])
print(output)
print(len(output))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CONTEXT: fore it crash, sky falling, Drop that ass, fore, higher, falling, Drop, crash, sky, ass, baby, switched my robot RAP-LYRICS: 
You wanna be on top
But I
160


## Data Doc

- Config 1
5 Epochs and substituting '\n' characters to spaces, while keeping the prompt format `CONTEXT: {keywords}\nRAP-LYRICS: {lyrics}` leads to short output. However, the quality of the output is okish.

- Config 2
bos and eos are added to the tokenizer. Input is also modified to `<|startoftext|>CONTEXT: {keywords} RAP-LYRICS: {lyrics}<|endoftext|>\n`. Training with 5 Epochs.
  - `<|newline|>` character apperars. Text is still too short.
- Config 3
Config 2 tested again, reloaded, and with 3 epochs only.